In [1]:
from pyomo.environ import *

## An Easy Two-stage Stochastic Programming Example in Process Design


<img src="illustrations/so_reaction.png" alt="process" width="800" height="500"/>


### Nomenclature:

#### Decision Variables:

- $F_{A}$: flow rate of A (reactant)

- $F_{B}$: flow rate of B (reactant)

- $F_{C}$: flow rate of C (product)

- $F_{D}$: flow rate of D (product)

- $R_{C}$: backorder C

- $R_{C}$: backorder D

- $y_1$: whether reaction 1 is selected.

- $y_2$: whether reaction 2 is selected.

#### Uncertain Parameters:

- $P_{C}, P_{D}$: Market prices for C,D at the end of the season (exogenous uncertainty).

- $D_{C}, D_{D}$: Demand for C, D

#### Deterministic parameters

- $\xi_{1}, \xi_{2}$: Yields for reaction 1 and 2

- $C^{v}_{A}, C^{v}_{B}$: unit cost of A and B, respectively.

- $C^{f}_{1}, C^{f}_{2}$: fixed cost of reactor 1 and 2, respectively.

- $Q$: Processing capacity for the reactor.

- $P^{B}_{C}, P^{B}_{D}$: Backorder price for C and D.

### Deterministic Formulation

#### Constraints:

1 **Capacity Constraints**:
\begin{align*}
F_{A} \le Q \cdot y_{1} \\
F_{B} \le Q \cdot y_{2}
\end{align*}

2 **Yield Constraints**:

\begin{align*}
\xi_{1} F_{A} = F_{C} \\
\xi_{2} F_{B} = F_{D}
\end{align*}

3 **Demand/backorder Constraint**:

\begin{align*}
F_{C} + R_{C} >= D_{C} \\
F_{D} + R_{D} >= D_{D}
\end{align*}

#### Objective:

$$ \textbf{min} \quad (C^{f}_{1} y_1 + C^{f}_{2}y_2) + (C^{v}_{A}F_{A} + C^{v}_{B}F_{B}) + ( P^{B}_{C}R_{C} + P^{B}_{D} R_{D} - P_{C}F_{C} - P_{D}F_{D})  $$

In [2]:
def build_det():
    m = ConcreteModel()

    m.Q = Param(initialize=20)
    m.xi1 = Param(initialize=0.8)
    m.xi2 = Param(initialize=0.4)
    m.cf1 = Param(initialize=20)
    m.cf2 = Param(initialize=50)
    m.cva = Param(initialize=2)
    m.cvb = Param(initialize=4)
    m.pbc = Param(initialize=20)
    m.pbd = Param(initialize=25)

    m.pc = Param(initialize=10)
    m.pd = Param(initialize=12)
    m.Dc = Param(initialize=16)
    m.Dd = Param(initialize=2)

    m.Fa = Var(domain=NonNegativeReals)
    m.Fb = Var(domain=NonNegativeReals)
    m.Fc = Var(domain=NonNegativeReals)
    m.Fd = Var(domain=NonNegativeReals)

    m.y1 = Var(domain=Binary)
    m.y2 = Var(domain=Binary)

    m.Rc = Var(domain=NonNegativeReals)
    m.Rd = Var(domain=NonNegativeReals)

    def _c1a(m):
        return m.Fa <= m.Q * m.y1
    m.c1a = Constraint(rule=_c1a)

    def _c1b(m):
        return m.Fb <= m.Q * m.y2
    m.c1b = Constraint(rule=_c1b)

    def _c2a(m):
        return m.xi1 * m.Fa == m.Fc
    m.c2a = Constraint(rule=_c2a)

    def _c2b(m):
        return m.xi2 * m.Fb == m.Fd
    m.c2b = Constraint(rule=_c2b)

    def _c3c(m):
        return m.Fc + m.Rc >= m.Dc
    m.c3c = Constraint(rule=_c3c)

    def _c3d(m):
        return m.Fd + m.Rd >= m.Dd
    m.c3d = Constraint(rule=_c3d)

    m.obj = Objective(
        expr = (m.cf1*m.y1 + m.cf2*m.y2) + (m.cva*m.Fa + m.cvb*m.Fb)
               + (m.pbc*m.Rc + m.pbd*m.Rd - m.pc*(m.Fc+m.Rc) - m.pd*(m.Fd+m.Rd)),
        sense=minimize
    )

    return m



In [3]:
mdet = build_det()

opt1 = SolverFactory('gurobi_persistent')
opt1.set_instance(mdet)
opt1.solve();

Restricted license - for non-production use only - expires 2025-11-24


In [4]:
for v in mdet.component_objects(Var, active=True):
    v.pprint()
value(mdet.obj)

Fa : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :  20.0 :  None : False : False : NonNegativeReals
Fb : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   0.0 :  None : False : False : NonNegativeReals
Fc : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :  16.0 :  None : False : False : NonNegativeReals
Fd : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   0.0 :  None : False : False : NonNegativeReals
y1 : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   1.0 :     1 : False : False : Binary
y2 : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   0.0 :     1 : False : False : Binary
Rc : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   0.0 :  None : Fal

-74.0

### Stochastic formulation

Three scenarios with equal probability $\Psi(\omega)$.

First stage variables:

Group 1: discrete
$$ y^{\omega}_{1}, y^{\omega}_{2} $$

Group 2: continuous
$$ F^{\omega}_{A}, F^{\omega}_{B}, F^{\omega}_{C}, F^{\omega}_{D} $$

Recourse actions:
$$ R^{\omega}_{C}, R^{\omega}_{D} $$

#### Constraints:

1 **Capacity Constraints**:
\begin{align*}
F^{\omega}_{A} \le Q \cdot y^{\omega}_{1} \\
F^{\omega}_{B} \le Q \cdot y^{\omega}_{2}
\end{align*}

2 **Yield Constraints**:

\begin{align*}
\xi_{1} F^{\omega}_{A} = F^{\omega}_{C} \\
\xi_{2} F^{\omega}_{B} = F^{\omega}_{D}
\end{align*}

3 **Demand/backorder Constraint**:

\begin{align*}
F^{\omega}_{C} + R^{\omega}_{C} \ge D^{\omega}_{C} \\
F^{\omega}_{D} + R^{\omega}_{D} \ge D^{\omega}_{D}
\end{align*}

4 **NACs**:

\begin{align*}
    x_{\omega} = x_{1}
\end{align*}

#### Objective:

$$ \textbf{min} \quad \sum_{\omega} \Psi^{\omega} ((C^{f}_{1} y^{\omega}_1 + C^{f}_{2}y^{\omega}_2) + (C^{v}_{A}F^{\omega}_{A} + C^{v}_{B}F^{\omega}_{B}) + ( P^{B}_{C}R^{\omega}_{C} + P^{B}_{D} R^{\omega}_{D} - P^{\omega}_{C}F^{\omega}_{C} - P^{\omega}_{D}F^{\omega}_{D}))  $$


In [5]:
def build_sp():
    m = ConcreteModel()

    # scenarios
    m.omega = Set(initialize=[1,2,3])

    # deterministic parameter
    m.Q = Param(initialize=20)
    m.xi1 = Param(initialize=0.8)
    m.xi2 = Param(initialize=0.4)
    m.cf1 = Param(initialize=20)
    m.cf2 = Param(initialize=50)
    m.cva = Param(initialize=2)
    m.cvb = Param(initialize=4)
    m.pbc = Param(initialize=20)
    m.pbd = Param(initialize=25)

    # uncertain parameter
    m.pc = Param(m.omega, initialize={1:5, 2:10, 3:15})
    m.pd = Param(m.omega, initialize={1:6, 2:12, 3:18})
    m.Dc = Param(m.omega, initialize={1:10, 2:16, 3:25})
    m.Dd = Param(m.omega, initialize={1:0, 2:2, 3:16})

    # Probability of scenario omega
    m.Psi = Param(m.omega, initialize={1:1/3, 2:1/3, 3:1/3})

    # First stage, continuous
    m.Fa = Var(m.omega, domain=NonNegativeReals)
    m.Fb = Var(m.omega, domain=NonNegativeReals)
    m.Fc = Var(m.omega, domain=NonNegativeReals)
    m.Fd = Var(m.omega, domain=NonNegativeReals)

    # First stage, discrete
    m.y1 = Var(m.omega, domain=Binary)
    m.y2 = Var(m.omega, domain=Binary)

    # Recourse action
    m.Rc = Var(m.omega, domain=NonNegativeReals)
    m.Rd = Var(m.omega, domain=NonNegativeReals)

    def nac_Fa(m, omega):
        if omega == m.omega.first():
            return Constraint.Skip
        return m.Fa[omega] == m.Fa[m.omega.first()]
    m.NAC_Fa = Constraint(m.omega, rule=nac_Fa)

    def nac_Fb(m, omega):
        if omega == m.omega.first():
            return Constraint.Skip
        return m.Fb[omega] == m.Fb[m.omega.first()]
    m.NAC_Fb = Constraint(m.omega, rule=nac_Fb)

    def nac_Fc(m, omega):
        if omega == m.omega.first():
            return Constraint.Skip
        return m.Fc[omega] == m.Fc[m.omega.first()]
    m.NAC_Fc = Constraint(m.omega, rule=nac_Fc)

    def nac_Fd(m, omega):
        if omega == m.omega.first():
            return Constraint.Skip
        return m.Fd[omega] == m.Fd[m.omega.first()]
    m.NAC_Fd = Constraint(m.omega, rule=nac_Fd)

    def nac_y1(m, omega):
        if omega == m.omega.first():
            return Constraint.Skip
        return m.y1[omega] == m.y1[m.omega.first()]
    m.NAC_y1 = Constraint(m.omega, rule=nac_y1)

    def nac_y2(m, omega):
        if omega == m.omega.first():
            return Constraint.Skip
        return m.y2[omega] == m.y2[m.omega.first()]
    m.NAC_y2 = Constraint(m.omega, rule=nac_y2)

    def _c1a(m, omega):
        return m.Fa[omega] <= m.Q * m.y1[omega]
    m.c1a = Constraint(m.omega, rule=_c1a)

    def _c1b(m, omega):
        return m.Fb[omega] <= m.Q * m.y2[omega]
    m.c1b = Constraint(m.omega, rule=_c1b)

    def _c2a(m, omega):
        return m.xi1 * m.Fa[omega] == m.Fc[omega]
    m.c2a = Constraint(m.omega, rule=_c2a)

    def _c2b(m, omega):
        return m.xi2 * m.Fb[omega] == m.Fd[omega]
    m.c2b = Constraint(m.omega, rule=_c2b)

    def _c3c(m, omega):
        return m.Fc[omega] + m.Rc[omega] >= m.Dc[omega]
    m.c3c = Constraint(m.omega, rule=_c3c)

    def _c3d(m, omega):
        return m.Fd[omega] + m.Rd[omega] >= m.Dd[omega]
    m.c3d = Constraint(m.omega, rule=_c3d)

    m.obj = Objective(
        expr = sum(((m.cf1*m.y1[omega] + m.cf2*m.y2[omega]) + (m.cva*m.Fa[omega] + m.cvb*m.Fb[omega])
                    + (m.pbc*m.Rc[omega] + m.pbd*m.Rd[omega] - m.pc[omega]*(m.Fc[omega]+m.Rc[omega])
                       - m.pd[omega]*(m.Fd[omega]+m.Rd[omega]))) * m.Psi[omega] for omega in m.omega),
        sense=minimize
    )

    return m

In [6]:
msp = build_sp()

opt2 = SolverFactory('gurobi_persistent')
opt2.set_instance(msp)
opt2.solve();

In [7]:
for v in msp.component_objects(Var, active=True):
    v.pprint()
value(msp.obj)

Fa : Size=3, Index=omega
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  20.0 :  None : False : False : NonNegativeReals
      2 :     0 :  20.0 :  None : False : False : NonNegativeReals
      3 :     0 :  20.0 :  None : False : False : NonNegativeReals
Fb : Size=3, Index=omega
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :  None : False : False : NonNegativeReals
      2 :     0 :   0.0 :  None : False : False : NonNegativeReals
      3 :     0 :   0.0 :  None : False : False : NonNegativeReals
Fc : Size=3, Index=omega
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  16.0 :  None : False : False : NonNegativeReals
      2 :     0 :  16.0 :  None : False : False : NonNegativeReals
      3 :     0 :  16.0 :  None : False : False : NonNegativeReals
Fd : Size=3, Index=omega
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :  None : False : False : NonNegativeReals
  

-38.99999999999999

In [8]:
def build_sp2():
    m = ConcreteModel()

    # scenarios
    m.omega = Set(initialize=[1,2,3])

    # deterministic parameter
    m.Q = Param(initialize=20)
    m.xi1 = Param(initialize=0.8)
    m.xi2 = Param(initialize=0.4)
    m.cf1 = Param(initialize=20)
    m.cf2 = Param(initialize=50)
    m.cva = Param(initialize=2)
    m.cvb = Param(initialize=4)
    m.pbc = Param(initialize=20)
    m.pbd = Param(initialize=25)

    # uncertain parameter
    m.pc = Param(m.omega, initialize={1:5, 2:10, 3:15})
    m.pd = Param(m.omega, initialize={1:6, 2:12, 3:18})
    m.Dc = Param(m.omega, initialize={1:10, 2:16, 3:25})
    m.Dd = Param(m.omega, initialize={1:0, 2:2, 3:16})

    # Probability of scenario omega
    m.Psi = Param(m.omega, initialize={1:1/3, 2:1/3, 3:1/3})

    # First stage, continuous
    m.Fa = Var(m.omega, domain=NonNegativeReals)
    m.Fb = Var(m.omega, domain=NonNegativeReals)
    m.Fc = Var(m.omega, domain=NonNegativeReals)
    m.Fd = Var(m.omega, domain=NonNegativeReals)

    # First stage, discrete
    m.y1 = Var(m.omega, domain=Binary)
    m.y2 = Var(m.omega, domain=Binary)

    # Recourse action
    m.Rc = Var(m.omega, domain=NonNegativeReals)
    m.Rd = Var(m.omega, domain=NonNegativeReals)

    def nac_y1(m, omega):
        if omega == m.omega.first():
            return Constraint.Skip
        return m.y1[omega] == m.y1[m.omega.first()]
    m.NAC_y1 = Constraint(m.omega, rule=nac_y1)

    def nac_y2(m, omega):
        if omega == m.omega.first():
            return Constraint.Skip
        return m.y2[omega] == m.y2[m.omega.first()]
    m.NAC_y2 = Constraint(m.omega, rule=nac_y2)

    def _c1a(m, omega):
        return m.Fa[omega] <= m.Q * m.y1[omega]
    m.c1a = Constraint(m.omega, rule=_c1a)

    def _c1b(m, omega):
        return m.Fb[omega] <= m.Q * m.y2[omega]
    m.c1b = Constraint(m.omega, rule=_c1b)

    def _c2a(m, omega):
        return m.xi1 * m.Fa[omega] == m.Fc[omega]
    m.c2a = Constraint(m.omega, rule=_c2a)

    def _c2b(m, omega):
        return m.xi2 * m.Fb[omega] == m.Fd[omega]
    m.c2b = Constraint(m.omega, rule=_c2b)

    def _c3c(m, omega):
        return m.Fc[omega] + m.Rc[omega] >= m.Dc[omega]
    m.c3c = Constraint(m.omega, rule=_c3c)

    def _c3d(m, omega):
        return m.Fd[omega] + m.Rd[omega] >= m.Dd[omega]
    m.c3d = Constraint(m.omega, rule=_c3d)

    m.obj = Objective(
        expr = sum(((m.cf1*m.y1[omega] + m.cf2*m.y2[omega]) + (m.cva*m.Fa[omega] + m.cvb*m.Fb[omega])
                    + (m.pbc*m.Rc[omega] + m.pbd*m.Rd[omega] - m.pc[omega]*(m.Fc[omega]+m.Rc[omega])
                       - m.pd[omega]*(m.Fd[omega]+m.Rd[omega]))) * m.Psi[omega] for omega in m.omega),
        sense=minimize
    )

    return m

In [9]:
msp2 = build_sp2()

opt3 = SolverFactory('gurobi_persistent')
opt3.set_instance(msp2)
opt3.solve();

In [10]:
for v in msp2.component_objects(Var, active=True):
    v.pprint()
value(msp2.obj)

Fa : Size=3, Index=omega
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  20.0 :  None : False : False : NonNegativeReals
      2 :     0 :  20.0 :  None : False : False : NonNegativeReals
      3 :     0 :  20.0 :  None : False : False : NonNegativeReals
Fb : Size=3, Index=omega
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :  None : False : False : NonNegativeReals
      2 :     0 :  20.0 :  None : False : False : NonNegativeReals
      3 :     0 :  20.0 :  None : False : False : NonNegativeReals
Fc : Size=3, Index=omega
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  16.0 :  None : False : False : NonNegativeReals
      2 :     0 :  16.0 :  None : False : False : NonNegativeReals
      3 :     0 :  16.0 :  None : False : False : NonNegativeReals
Fd : Size=3, Index=omega
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :  None : False : False : NonNegativeReals
  

-43.0